<div style=width:65%>

# Sprite Group Collisions
Let's say you're creating a classic space shooter like the one shown below:<br>

<img src="https://img.itch.zone/aW1nLzE1NjAyMDMucG5n/original/fD6xTJ.png" width=500/><br>

Your ship fires bullets at enemy ships and you need to check to see if the bullets collide with the enemy ships. Let's say that we have the following sprite class: Enemy, Player and Bullet.<br>
<ol>
<li>Create an Enemy sprite group</li>
<li>Create a Bullet sprite group</li>
</ol>

##### <span style="color:green">Collision Checking</span>

We need to take advantage of the groups we placed sprites into to check for collisions. The <code>pygame.sprite.spritecollide(\<sprite>, \<group>, \<do kill>)</code><br>

<ul>
<li>sprite - main sprite compared in collisions</li>
<li>group - all sprites in this group are checked for collision with main sprite</li>
<li>do kill - this is a boolean; if true, the sprite from the group is removed from the group</li>
</ul>

<span style="color:red">Warning:</span> For the following example, be sure to include <span style="color:cyan">03 - ghost.png</span> and <span style="color:cyan">03 - pacman.png</span> into the same folder as this lesson file.
</div>

In [ ]:
import pygame, sys
from config import *
import random

#creates a sprite with a pacman image
class Pacman(pygame.sprite.Sprite):
    def __init__(self, position):
        super().__init__()
        self.image = pygame.image.load("03 - pacman.png")
        self.rect = self.image.get_rect()
        self.rect.topleft = position
        self.speed = random.randint(2, 5) #chooses value between 2 and 5 for speed once

    #moves pacman left and right
    def update(self):
        self.rect.topleft += pygame.Vector2(self.speed, 0)
        if self.rect.left <= 0:
            self.speed *= -1
        elif self.rect.right >= WIDTH:
            self.speed *= -1

#creates a sprite with a ghost image
class Ghost(pygame.sprite.Sprite):
    def __init__(self, position):
        super().__init__()
        self.image = pygame.image.load("03 - ghost.png")
        self.rect = self.image.get_rect()
        self.rect.topleft = position
        self.speed = random.randint(1, 3) #chooses value between 1 and 3 for speed once

    #moves ghost up and down
    def update(self):
        self.rect.topleft += pygame.Vector2(0, self.speed)
        if self.rect.top <= 0:
            self.speed *= -1
        elif self.rect.bottom >= HEIGHT:
            self.speed *= -1

class Game:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()

        #creates 2 pacman sprites and adds to pacmanGroup
        pacman = Pacman((WIDTH/2, HEIGHT/4))
        pacman2 = Pacman((WIDTH/2, HEIGHT*3/4))
        self.pacmanGroup = pygame.sprite.Group()
        self.pacmanGroup.add(pacman, pacman2)

        #creates 4 to 20 ghost sprites and adds to ghostGroup
        self.ghostGroup = pygame.sprite.Group()
        for i in range(random.randint(4, 20)):
            pos = (random.randint(0, WIDTH), 10)
            g = Ghost(pos)
            self.ghostGroup.add(g)

    #removes ghost sprites from the ghostGroup if they collide with a pacman sprite
    def checkCollisions(self):
        if self.pacmanGroup: #True if there are pacman sprites in the group, false otherwise
            for pacman in self.pacmanGroup:
                collided = pygame.sprite.spritecollide(pacman, self.ghostGroup, True)

    def run(self):
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                
            self.screen.fill("black")

            #updates and draws the various groups
            self.pacmanGroup.update()
            self.pacmanGroup.draw(self.screen)
            self.ghostGroup.update()
            self.ghostGroup.draw(self.screen)

            self.checkCollisions() #self-explanatory

            pygame.display.update()
            self.clock.tick(FPS)
    
if __name__ == "__main__":
    game = Game()
    game.run()

<div style=width:65%>

In the next example, we'll set the <code>do kill</code> argument in <code>pygame.sprite.spritecollide()</code> to False. This means that when there's a collision it will not automatically remove the sprite from the group. In your game, you may want to do other things. For example, we'll shrink the pacman sprites while they are touching the ghost sprites. Once their size becomes 0, we'll use the <code>\<sprite>.kill()</code> method to remove :
</div>

In [ ]:
import pygame, sys
from config import *
import random

class Pacman(pygame.sprite.Sprite):
    def __init__(self, position):
        super().__init__()
        self.image = pygame.image.load("03 - pacman.png")
        self.rect = self.image.get_rect()
        self.rect.topleft = position
        print(position)
        self.speed = random.randint(2, 5)

    def update(self):
        self.rect.topleft += pygame.Vector2(self.speed, 0)
        if self.rect.left <= 0:
            self.speed *= -1
        elif self.rect.right >= WIDTH:
            self.speed *= -1

class Ghost(pygame.sprite.Sprite):
    def __init__(self, position):
        super().__init__()
        self.image = pygame.image.load("03 - ghost.png")
        self.rect = self.image.get_rect()
        self.rect.topleft = position
        print(position)
        self.speed = random.randint(1, 3)

    def update(self):
        self.rect.topleft += pygame.Vector2(0, self.speed)
        if self.rect.top <= 0:
            self.speed *= -1
        elif self.rect.bottom >= HEIGHT:
            self.speed *= -1

class Game:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()

        pacman = Pacman((WIDTH/2, HEIGHT/4))
        pacman2 = Pacman((WIDTH/2, HEIGHT*3/4))
        self.pacmanGroup = pygame.sprite.Group()
        self.pacmanGroup.add(pacman, pacman2)

        #creates 10 to 30 ghost sprites and adds to ghostGroup
        self.ghostGroup = pygame.sprite.Group()
        for i in range(random.randint(10, 30)):
            pos = (random.randint(0, WIDTH), 10)
            g = Ghost(pos)
            self.ghostGroup.add(g)

    #shrinks ghosts while they are touching pacman sprites and removes from the group once
    #the height or width becomes zero via the <sprite>.kill() method
    def checkCollisions(self):
        if self.pacmanGroup:
            for pacman in self.pacmanGroup:
                collided = pygame.sprite.spritecollide(pacman, self.ghostGroup, False)

                for ghost in collided:
                    x,y,w,h = ghost.image.get_rect()
                    ghost.image = pygame.transform.scale(ghost.image, (w*0.99,h*0.99))
                    if w == 0 or h == 0:
                        ghost.kill() #removes the ghost from its group

    def run(self):
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                
            self.screen.fill("black")

            #updates and draws the various groups
            self.pacmanGroup.update()
            self.pacmanGroup.draw(self.screen)
            self.ghostGroup.update()
            self.ghostGroup.draw(self.screen)

            self.checkCollisions()

            pygame.display.update()
            self.clock.tick(FPS)
    
if __name__ == "__main__":
    game = Game()
    game.run()